# LSTM RNN

Build a Network

In [ ]:
# keras:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv1D, LSTM, Input, ReLU
from keras.layers import Flatten, Dropout, Reshape, ZeroPadding1D, Cropping1D
from keras.layers import GlobalAveragePooling1D, AveragePooling1D, UpSampling1D, MaxPooling1D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.utils import plot_model
from keras import models

from sklearn.model_selection import train_test_split
import time

In [ ]:
def training(model, input, output, batch=64, epochs=20,lr=False):
  if type(lr) != bool:
    K.set_value(model.optimizer.learning_rate, lr)
  history = model.fit(input,output,batch,epochs,validation_split=0.1,verbose=1,
                  callbacks = [ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-7)])
  return model, history

  # EarlyStopping(monitor='val_loss', patience=25),
          # ModelCheckpoint(filepath=filePath, monitor='val_loss', save_best_only=True),

In [ ]:
# define percent error
def percent_error(data_test,data_predict,n_points):
  return 100*np.mean(np.abs(data_test-data_predict)/data_test,axis=1)

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(c_train, s_train, test_size=0.15)

In [ ]:
# Building Sequential Model

Net=Sequential()
Net.add(LSTM(250,input_shape=(150,1),return_sequences=True))
Net.add(LSTM(150,return_sequences=False))
Net.add(Dense(200,activation="relu"))
Net.add(Reshape((200,1)))
# Net.add(LSTM(60,return_sequences=True))
Net.add(LSTM(150,return_sequences=False))
# Net.add(LSTM(250,return_sequences=False))
# Net.add(Dropout(0.03))
# Net.add(LSTM(150,return_sequences=False))
Net.add(Dense(250,activation="relu"))
Net.add(Dense(501,activation="exponential"))
Net.compile(loss='MAPE',optimizer='adam')
Net.summary()

In [ ]:
trainable_lstm = np.sum([K.count_params(w) for w in Net.trainable_weights])
trainable_lstm

In [ ]:
y_train.shape

Train the network

In [ ]:
n_epochs = 200
start = time.time()
lstm, history_lstm = training(Net,x_train,y_train,batch=64,epochs=n_epochs,
                              lr=0.001,
                              )
end = time.time()
time_lstm = end - start
print(time_lstm)

In [ ]:
# Saving the trained model
lstm.save(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep200_X32', overwrite=True)

# Saving the training history
pd.DataFrame(history_lstm.history).to_csv(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep200_x32_history'+'.csv')

In [ ]:
# y_predict = Net.predict(x_test)
y_predict = loaded_model.predict(x_test)
err_predict = percent_error(y_test,y_predict,501)

In [ ]:
np.mean(err_predict)

In [ ]:
hist_lstm,x =np.histogram(err_predict,bins=51,range=[0,50],density=False)
x = (x[:-1]+x[1:])/2

In [ ]:
n_plot = 5
rand_set = np.random.randint(0,y_test.shape[0],(n_plot,))

In [ ]:
color_str = ["#d9ed92","#b5e48c","#99d98c","#76c893","#52b69a","#34a0a4","#168aad","#1a759f","#1e6091","#184e77"]
fig = make_subplots(rows=1, cols=3, subplot_titles=('Training History - LSTM','Mean Absolute Error', 'Comparison'))

fig.add_scatter(x=np.arange(1,301),y=history_lstm.history['val_loss'],line=dict(width=3,color='royalblue'),opacity=1,name=f'Validation loss',row=1,col=1,showlegend=False)
fig.add_trace(go.Bar(x=x,y=hist_lstm,opacity=0.75,name='Error',showlegend=False),row=1,col=2)
for i in range(n_plot):
  # fig.add_scatter(x=t*1e6,y=x_test[rand_set[i],:],line=dict(width=2,color=color_str[i]),opacity=1,name=f'Input-{rand_set[i]}',row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_test[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Expected-{rand_set[i]}',showlegend=True,row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_predict[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Predicted-{rand_set[i]}',showlegend=True,row=1,col=3)

fig.update_layout(template=fig_template,
                  width = 1400,
                  xaxis1 = dict(title='Epochs'),
                  yaxis1 = dict(title='Mean absolute error on validation set',type="log",),
                  xaxis2 = dict(title='% Error',range=[0,50]),
                  yaxis2 = dict(title='Samples'),
                  xaxis3 = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                              #  type="log",
                              #  range=[,],
                               ),
                  yaxis3 = dict(title='S(\N{greek small letter omega})',
                              #  range=[-0.1e6,0.1e6],
                              #  type="log",

                               ),

                 )

In [ ]:
loaded_model = models.load_model(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep200_X32')
# history_stored = pd.read_csv(path+'/cnn_trial'+'.csv')

In [ ]:
loaded_model_cnn = models.load_model("/content/gdrive/My Drive/Research/ML_PROJECTS_ON_SUPERCONDUCTING_QUBITS"+'/CNN/TRAINED_NETWORKS/MODEL_7.36_fil=32_ker=21_dr0.05_ps=2_LRini=0.001_LRmin=1e-05_bs=64_ep=200_nt=5')

Predictions

In [ ]:
loaded_model0 = models.load_model(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep400')
loaded_model1 = models.load_model(trainData_path+'/LSTM_250_150_d200_150_d250_d501_ep200')

In [ ]:
# (0,1,2,9,10,11,12,13,14,18,19,20)
select_data = norm_data[(1,14),:]
exp_predict0 = loaded_model0.predict(select_data)
exp_predict1 = loaded_model1.predict(select_data)

In [ ]:
test_predict = loaded_model.predict(data['c_expt'])

In [ ]:
test_predict_cnn = loaded_model_cnn.predict(data['c_expt'])

In [ ]:
test_predict_cnn.shape

In [ ]:
## To extend the noise spectrum to lower frequencies

def spectrum_extend(exp_predict,w_train,w_new):
  w_lowSize = np.argwhere(w_new<w_train.min()).size
  w_lowArg = np.argwhere(w_new<w_train.min())[0]

  s_extend = np.zeros((exp_predict.shape[0],w_new.size))
  s_extend[:,int(w_lowArg):] = np.repeat(np.mean(exp_predict[:,-4:],axis=1),
                                        int(w_lowSize)).reshape(exp_predict.shape[0],int(w_lowSize))
  for i in range(exp_predict.shape[0]):
    s_extend[i,:int(w_lowArg)] = interpData(w_train,exp_predict[i,:],w_new[:int(w_lowArg)])
  return s_extend

## Calculate noise using delta-function approximation for 32-pi-pulse sequence

def delta_noise(c, t):
  w_delta = 32*np.pi/t
  s_delta = -np.pi*np.log(c)/t
  return s_delta, w_delta

In [ ]:
w_train.min()

In [ ]:
# s_delta, w_delta = delta_noise(select_data,T_train)
# s_delta, w_delta = delta_noise(x_test,T_train)

w_new = np.flipud(np.geomspace(1e3,w_train.max(),1001))

s_extend = spectrum_extend(test_predict,w_train,w_new)
s_extend_cnn = spectrum_extend(test_predict_cnn,w_train,w_new)

c_predict = getCoherence(s_extend,w_new,T_train,32,48e-9)
c_predict_cnn = getCoherence(s_extend_cnn,w_new,T_train,32,48e-9)
# c_delta = getCoherence(s_delta,w_delta,T_train,32,48e-9)

In [ ]:
np.savez_compressed("/content/gdrive/My Drive/Research/ML_PROJECTS_ON_SUPERCONDUCTING_QUBITS"+'/CNN/TRAINED_NETWORKS/predictions_Mar13',
                    c_expt=data['c_expt'],t_expt=data['xval'],s_predict=s_extend_cnn,w_predict=w_new)

In [ ]:
w_new.shape

In [ ]:
fig = go.Figure()
color_str = ["#d9ed92","#b5e48c","#99d98c","#76c893","#52b69a","#34a0a4","#168aad","#1a759f","#1e6091","#184e77"]
n_plot = 10
rand_set = np.random.randint(0,data['c_expt'].shape[0],(n_plot,))
for j in range(n_plot):
  # print(j)
  # fig.add_scatter(x=T_train*1e6,y=x_test[rand_set[j],:],line=dict(width=4,color=color_str[j]),opacity=0.8,name=f'Expt {j}')
  fig.add_scatter(x=T_train*1e6,y=data['c_expt'][rand_set[j],:],line=dict(width=4,color=color_str[j]),opacity=0.8,name=f'Expt {j}')
  fig.add_scatter(x=T_train*1e6,y=c_predict[rand_set[j],:],line=dict(width=3,color=color_str[j]),opacity=1,name=f'Predict {j}')
  fig.add_scatter(x=T_train*1e6,y=c_predict_cnn[rand_set[j],:],line=dict(width=3,color=color_str[j]),opacity=1,name=f'Predict_cnn {j}')
  # fig.add_scatter(x=T_train*1e6,y=c_delta[rand_set[j],:],line=dict(width=2),opacity=0.8,name=f'Delta {j}')
fig.update_layout(template=fig_template,  title = 'Coherence Curves',
                  width = 600,
                  xaxis = dict(title='Evolution time (\N{greek small letter mu}s)',
                              #  range=[0,400],
                               ),
                  yaxis = dict(title='Coherence'),
                 )

In [ ]:
fig = go.Figure()
for i in range(select_data.shape[0]):
  # fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=exp_predict0[i,:],line=dict(width=3),opacity=0.8,showlegend=True,name=f'{i}')
  # fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=exp_predict1[i,:],line=dict(width=3),opacity=0.8,showlegend=True,name=f'{i}')
  fig.add_scatter(x=1e-6*w_new/(2*np.pi),y=s_extend[i,:],line=dict(width=3),opacity=1,showlegend=True,name=f'Predict. {i}')

fig.add_scatter(x=1e-6*w_delta/(2*np.pi),y=s_delta[1,:],line=dict(width=3,color='grey'),opacity=0.2,showlegend=True,name=f'Approx.')

fig.update_layout(template=fig_template,
                  width = 600,
                  xaxis = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                               type="log",
                              #  range=[,],
                               ),
                  yaxis = dict(title='S(\N{greek small letter omega})',
                               range=[3.6,4.6],
                               type="log",

                               ),

                 )

In [ ]:
fig = go.Figure()
for j in range(n_plot):
  # fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_test[rand_set[j],:],line=dict(width=3,color=color_str[j]),opacity=1,showlegend=True,name=f'Sim. {j}')
  # fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=test_predict[rand_set[j],:],line=dict(width=3,color=color_str[j]),opacity=1,showlegend=True,name=f'Pred. {j}')
  fig.add_scatter(x=1e-6*w_new/(2*np.pi),y=s_extend[rand_set[j],:],line=dict(width=3,color=color_str[j]),opacity=1,showlegend=True,name=f'Pred.Ex {j}')
  fig.add_scatter(x=1e-6*w_new/(2*np.pi),y=s_extend_cnn[rand_set[j],:],line=dict(width=3,color=color_str[j]),opacity=1,showlegend=True,name=f'Pred.Ex_cnn {j}')
  # fig.add_scatter(x=1e-6*w_delta/(2*np.pi),y=s_delta[rand_set[j],:],line=dict(width=3,color=color_str[j]),opacity=0.2,showlegend=True,name=f'Approx. {j}')

fig.update_layout(template=fig_template,
                  width = 600,
                  xaxis = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                               type="log",
                              #  range=[,],
                               ),
                  yaxis = dict(title='S(\N{greek small letter omega})',
                               range=[2.6,4.6],
                               type="log",

                               ),

                 )

# CNN 

In [ ]:
from tensorflow.keras import layers

Model network building

In [ ]:
filter_nb=32
kernel_size=32
dropout_rate=0.03
pool_size=2
xtrain_size = np.shape(x_train)[-1]
ytrain_size = np.shape(y_train)[-1]

model = models.Sequential()
model.add( layers.Input( shape=(xtrain_size, 1) ) )
model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
model.add( layers.Conv1D(filter_nb/2, kernel_size,activation="relu", padding='same' ) )
model.add( layers.Flatten() )
model.add( layers.Dense(501, activation='relu') )
model.add( layers.Reshape((501,1) ))
# model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
model.add( layers.Conv1D(filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.MaxPooling1D( pool_size=pool_size, padding="same") )
# model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.UpSampling1D( size=pool_size ) )
# model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.UpSampling1D( size=pool_size ) )
model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
# model.add( layers.UpSampling1D( size=pool_size ) )
# model.add( layers.Conv1D( filter_nb,kernel_size,activation="relu", padding='same' ) )
model.add( layers.Conv1D( 1, kernel_size, activation="relu", padding='same' ) )
model.add( layers.Flatten() )
# model.add( layers.Dense(250, activation='relu') )
model.add( layers.Dropout( dropout_rate ) )
model.add( layers.Dense(ytrain_size, activation='linear') )
model.compile(loss='MAPE',optimizer='adam')
model.summary()

In [ ]:
trainable_cnn = np.sum([K.count_params(w) for w in model.trainable_weights])
trainable_cnn

In [ ]:
n_epochs = 450
start = time.time()
cnn, history_cnn = training(model,x_train,y_train,batch=64,epochs=n_epochs,
                              lr=0.001,
                              )
end = time.time()
time_cnn = end - start
print(time_cnn)

In [ ]:
# Saving the trained model
cnn.save(trainData_path+'/cnn_noUpsampling', overwrite=True)

# Saving the training history
pd.DataFrame(history_cnn.history).to_csv(path+'/cnn_noUpsampling'+'.csv')

In [ ]:
loaded_model = models.load_model(trainData_path+'/cnn_noUpsampling')
history_stored = pd.read_csv(path+'/cnn_noUpsampling'+'.csv')

In [ ]:
y_predict = model.predict(x_test)
err_predict = percent_error(y_test,y_predict,501)

In [ ]:
np.mean(err_predict)

In [ ]:
history_stored

In [ ]:
hist_cnn,x =np.histogram(err_predict,bins=51,range=[0,50],density=False)
x = (x[:-1]+x[1:])/2

In [ ]:
n_plot = 5
rand_set = np.random.randint(0,y_test.shape[0],(n_plot,))

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Training History - CNN','Mean Absolute Error', 'Comparison'))
# history_cnn.history['val_loss']
fig.add_scatter(x=np.arange(1,301),y=history_cnn.history['val_loss'],line=dict(width=3,color='royalblue'),opacity=1,name=f'Validation loss',row=1,col=1,showlegend=False)
fig.add_trace(go.Bar(x=x,y=hist_cnn,opacity=0.75,name='Error',showlegend=False),row=1,col=2)
for i in range(n_plot):
  # fig.add_scatter(x=t*1e6,y=x_test[rand_set[i],:],line=dict(width=2,color=color_str[i]),opacity=1,name=f'Input-{rand_set[i]}',row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_test[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Expected-{rand_set[i]}',showlegend=True,row=1,col=3)
  fig.add_scatter(x=1e-6*w_train/(2*np.pi),y=y_predict[rand_set[i],:],line=dict(width=3,color=color_str[i]),opacity=0.8,name=f'Predicted-{rand_set[i]}',showlegend=True,row=1,col=3)

fig.update_layout(template=fig_template,
                  width = 1000,
                  xaxis1 = dict(title='Epochs'),
                  yaxis1 = dict(title='Mean absolute error on validation set',type="log",),
                  xaxis2 = dict(title='% Error',range=[0,50]),
                  yaxis2 = dict(title='Samples'),
                  xaxis3 = dict(title='\N{greek small letter omega}/2\N{greek small letter pi} (MHz)',
                              #  type="log",
                              #  range=[,],
                               ),
                  yaxis3 = dict(title='S(\N{greek small letter omega})',
                              #  range=[-0.1e6,0.1e6],
                              #  type="log",

                               ),

                 )